# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096791


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:37,  1.40s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:34,  1.32s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:22,  1.05it/s]

Rendering models:  23%|██▎       | 7/30 [00:05<00:16,  1.43it/s]

...

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  5.18it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  6.36it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  6.54it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.96it/s]

not-logged-in-808d8d7eb7071532ff8d    0.031508
jnarayanbvg                           0.000192
Jonas_Cross                           0.000452
thiagoxxxx                            0.000150
Kartoon1x                             0.000101
rsmith8235                            0.004844
lsautter                              0.000205
mr_willock                            0.028052
sxr1526                               0.000109
Planetme                              0.000466
acapirala                             0.000154
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000592
froggler                              0.001423
Lavadude                              0.010547
crush202020                           0.000254
MatheusSilva                          0.000490
not-logged-in-848a203976350ff114c0    0.000366
nicholas.                             2.439741
h4ranek                               0.000802
FR4N1O                                0.001763
tingard                               0.000080
DCCstudents  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())